# Aim of this notebook:
1.  Read the CSV file that was created in the first notebook to Pandas DataFrame

2. Create new columns to the dataframe and fill them with right values using regular expression

3. Identify missing values, clean and drop off data and colums that are no longer needed

4. Save the dataframe as a new CSV file

In [1]:
import pandas as pd
import numpy as np
import re
import requests
from bs4 import BeautifulSoup

In [2]:
df_raw = pd.read_csv("256_url.csv")

Split the column "Project Description" to 2 new columns and replace the unnecessary characters with empty character

In [3]:
# NO ERROR HANDLING ON THESE

# Split the "Project Description" column into two separate columns
df_raw[['ProjectDescription', 'ProjectLead']] = df_raw['Project Description'].str.split('Poroject Lead:', expand=True)

# Extract "ProjectDescription" to a new list
project_description_list = df_raw['ProjectDescription'].tolist()

# Extract "ProjectLead" to a new list
project_lead_list = df_raw['ProjectLead'].tolist()

# Replace the "\n" with " " in "ProjectLead"

df_raw["ProjectLead"] = df_raw["ProjectLead"].str.replace("\n","")

Keep the column "Country" and add 1 new column using regular expression

In [4]:
# NO ERROR HANDLING ON THESE (also, I did not include city names)

# Extract "Country" column
df_raw["Country"] = df_raw["Region"].str.extract("Country:\n(.*?)\n")

# Set "Region" column automatically
df_raw["Region"] = df_raw["Region"].str.extract("Region:\n(.*?)\n")

# Remove the "\n" characters from the new columns
df_raw["Country"] = df_raw["Country"].str.strip()
df_raw["Region"] = df_raw["Region"].str.strip()

Take a copy of the dataframe and continue with the copy

In [5]:
testi_3 = df_raw.copy()

Create a function to extract values using regular expressions. Create 5 new columns that gets their values from the column "Project Boundaries"

In [6]:
# Define a function to extract values using regular expressions
def extract_value(pattern, text):
    try:
        result = re.search(pattern, text).group(1)
        return result
    except AttributeError:
        return np.nan

# Create empty columns in the DataFrame
testi_3['StartTime'] = np.nan
testi_3['EndTime'] = np.nan
testi_3['Status'] = np.nan
testi_3['Budget'] = np.nan
testi_3['FundingSource'] = np.nan

# Iterate over rows and apply the extraction and conversion functions
for index, row in testi_3.iterrows():
    testi_3.at[index, 'StartTime'] = extract_value(r'Start - End:\n(.*?) -', row['Project Boundaries'])
    testi_3.at[index, 'EndTime'] = extract_value(r'Start - End:\n.*? - (.*?)\n', row['Project Boundaries'])
    testi_3.at[index, 'Status'] = extract_value(r'Project Status:\n(.*?)\n', row['Project Boundaries'])
    
    budget_str = extract_value(r'Project Budget:\n(.*?) €', row['Project Boundaries'])
    if pd.notna(budget_str) and budget_str.replace(".", "", 1).isdigit():
        testi_3.at[index, 'Budget'] = float(budget_str.replace(',', ''))
    
    funding_source = extract_value(r'Funding source:\n(.*)', row['Project Boundaries'])
    if pd.notna(funding_source):
        if isinstance(funding_source, str):
            testi_3.at[index, 'FundingSource'] = funding_source.strip()
        else:
            testi_3.at[index, 'FundingSource'] = str(funding_source)

Create a function to clean and convert datetime values. Create 1 new column that gets its values from Start and End times that are not null

In [7]:
def clean_and_convert_date(date_str):
    try:
        date = pd.to_datetime(date_str, format='%Y-%m-%d', errors='coerce')
        return date
    except ValueError:
        return pd.NaT  # Use pd.NaT to represent a missing date

# Apply the cleaning and conversion function to "StartDate" and "EndDate"
testi_3['StartTime'] = testi_3['StartTime'].apply(clean_and_convert_date)
testi_3['EndTime'] = testi_3['EndTime'].apply(clean_and_convert_date)

# Calculate the "ProjectLength" column
testi_3['ProjectLength'] = (testi_3['EndTime'] - testi_3['StartTime']).dt.days

Take a copy of the dataframe and continue with the copy

In [8]:
testi_4 = testi_3.copy()

Create 1 new column from the "Website" column. There is no error handling on this and the "trash_list" contains values, that will be not included to the final dataframe

In [9]:
# Split the "Website" column into two separate columns
testi_4[['Trash','Website']] = testi_4['Website'].str.split('Website:\n', expand=True)

trash_list = testi_4['Website'].tolist()
website_list = testi_4['Website'].tolist()

Take a copy of the dataframe and continue with the copy

In [10]:
testi_9 = testi_4.copy() 

Create 7 new columns that gets their values from column "Technological Information"

In [11]:
# Define your column names that are produced from "Technological Information" column
column_names_from_tech_info_column = [
    'Activities',
    'CO2 Source',
    'CCU Tecnology Category',
    'Facility stage',
    'Start TRL',
    'End TRL',
    'Info On Utilization'
]

# Initialize empty columns in the DataFrame
for col in column_names_from_tech_info_column:
    testi_9[col] = np.nan

# Iterate over rows and split the text
for index, row in testi_9.iterrows():
    text = row['Technological Information']
    
    # Use a regex pattern to match the key and its associated values
    pattern = f"({'|'.join(column_names_from_tech_info_column)}):\\n(.*?)(?=(?:{'|'.join(column_names_from_tech_info_column)}):\\n|\\Z)"
    matches = re.findall(pattern, text, re.DOTALL)
    
    # Fill the corresponding column with the extracted values
    for match in matches:
        key, value = match
        testi_9.at[index, key] = value

# Replace empty strings with NaN
testi_9.replace('', np.nan, inplace=True) 

Replace the unnecessary characters from the new columns created from the column "Technological Information"

In [12]:
testi_9["Activities"] = testi_9["Activities"].str.replace('-','').str.replace('\n\n\n','').str.replace('\n',',').str.strip()
testi_9["CO2 Source"] = testi_9["CO2 Source"].str.replace('-','').str.replace('\n\n\n','').str.replace('\n',',').str.strip()
testi_9["CCU Tecnology Category"] = testi_9["CCU Tecnology Category"].str.replace('-','').str.replace('\n\n\n','').str.replace('\n',',').str.strip()
testi_9["Facility stage"] = testi_9["Facility stage"].str.replace('-','').str.replace('\n\n\n','').str.replace('\n',',').str.strip()
testi_9["Start TRL"] = testi_9["Start TRL"].str.replace('-','').str.replace('\n\n\n','').str.replace('\n',',').str.strip()
testi_9["End TRL"] = testi_9["End TRL"].str.replace('-','').str.replace('\n\n\n','').str.replace('\n',',').str.strip()
testi_9["Info On Utilization"] = testi_9["Info On Utilization"].str.replace('-','').str.replace('\n\n\n','').str.replace('\n',',').str.strip()

Take a copy of the dataframe and continue with the copy

In [13]:
testi_10 = testi_9.copy()

Create 2 new columns that gets their values from column "Technological Details"

In [14]:
# Define column names that are produced from "Technological Details" column

column_names_tech_details = [
    'CO2 capture/Utilization',
    'Production Volume'
]

# Initialize empty columns in the DataFrame
for col in column_names_tech_details:
    testi_10[col] = np.nan

# Iterate over rows and split the text
for index, row in testi_10.iterrows():
    text = row['Technological Details']
    
    # Use a regex pattern to match the key and its associated values
    pattern = f"({'|'.join(column_names_tech_details)}):\\n(.*?)(?=(?:{'|'.join(column_names_tech_details)}):\\n|\\Z)"
    matches = re.findall(pattern, text, re.DOTALL)
    
    # Fill the corresponding column with the extracted values
    for match in matches:
        key, value = match
        testi_10.at[index, key] = value

# Replace empty strings with NaN
testi_10.replace('', np.nan, inplace=True) 

Replace the unnecessary characters from the new columns created from the column "Technological Details"

In [15]:
testi_10["CO2 capture/Utilization"] = testi_10["CO2 capture/Utilization"].str.replace('\n\n','').str.replace('Production Volume:','').str.replace('\n\n\n','').str.strip()
testi_10["CO2 capture/Utilization"] = np.where(testi_10["CO2 capture/Utilization"] == "", np.nan, testi_10["CO2 capture/Utilization"])

Take a copy of the dataframe and continue with the copy

In [16]:
testi_11 = testi_10.copy()

Create 2 new columns that get their values from column "Product Information". This column was most challenging to chop into new columns and needed some extra mofication

In [17]:
# Define column names that are produced from "Product Information" column
column_names_product_information = [
    'Product Categories',
    'Specific Product(s)'
]

# Initialize empty columns in the DataFrame
for col in column_names_product_information:
    testi_11[col] = np.nan

# Iterate over rows and split the text
for index, row in testi_11.iterrows():
    text = row['Product Information']

    # Use a regex pattern to match the key and its associated values
    pattern = r"(.+?):\n(.*?)(?=(?:\n.+?:\n|$))"
    matches = re.findall(pattern, text, re.DOTALL)

    # Fill the corresponding column with the extracted values
    for key, value in matches:
        testi_11.at[index, key] = value

# Replace empty strings with NaN
testi_11.replace('', np.nan, inplace=True)

Replace the unnecessary characters from the new columns created from the column "Product Information"

In [18]:
testi_11["Product Categories"] = testi_11["Product Categories"].str.replace('-','').str.replace('Specific Product(s):','').str.replace('\n\n\n','').str.strip()
testi_11 = testi_11.rename(columns={"\n\n\nSpecific Product(s)": 'SpecificProductsA'})
testi_11 = testi_11.rename(columns={"\n- Chemicals\n- Biomass\n\n\nSpecific Product(s)": 'SpecificProductsB'})

C:\Users\OMISTAJA\AppData\Local\Temp\ipykernel_20452\209702105.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  testi_11["Product Categories"] = testi_11["Product Categories"].str.replace('-','').str.replace('Specific Product(s):','').str.replace('\n\n\n','').str.strip()


Take a copy of the dataframe and continue with the copy

In [19]:
testi_12 = testi_11.copy()

"Partners" column needed only replacing the not needed characters and it could also keep its original column name

In [20]:
testi_12["Partners"] = testi_12["Partners"].str.replace('-','').str.replace('\n',',').str.strip()

To get the projects names, I created another CO2 Values Europe webpage scraper. This scraper uses GET method and BeatifulSoup and reads the project names from the firts layer of the database and can change the pages of the webpage. 

In [21]:
url = "https://database.co2value.eu/"
data_titles = []  # Initialize this list

df = pd.DataFrame(columns=data_titles)

while True:
    page = requests.get(url)
    soup = BeautifulSoup(page.text, "html.parser")

    table = soup.find('table')
    data_titles_raw = soup.find_all('th')
    
    if not data_titles:
        data_titles = [title.text.strip() for title in data_titles_raw]
        df = pd.DataFrame(columns=data_titles)

    column_data = table.find_all('tr')

    for row in column_data[1:]:
        row_data = row.find_all('td')
        individual_row_data = [data.text.strip() for data in row_data]
        
        if individual_row_data:
            df.loc[df.shape[0]] = individual_row_data

    # Find the next page link
    pagination = soup.find('ul', class_='pagination')
    next_page_link = pagination.find('a', rel='next')

    if next_page_link:
        url = "https://database.co2value.eu/" + next_page_link['href']
    else:
        break  # Exit the loop if there's no next page

After the projects names are read to dataframe "df", 2 new columns are made

In [22]:
# Split the "Project" column into two separate columns
df[['ProjectName', 'ProjectInfo']] = df['Project'].str.split('\n\n\n', expand=True)

project_name_list = df['ProjectName'].tolist()

project_info_list = df['ProjectInfo'].tolist()

But because we already have values for project info, we include only the "ProjectName" to the dataframe we are working with

In [23]:
testi_12["ProjectName"] = project_name_list

Now we a first draft of the dataframe that we want to save as a CSV file

In [24]:
testi_12.head(3)

,Project Description,Region,Project Boundaries,Website,Technological Information,Technological Details,Product Information,Partners,ProjectDescription,ProjectLead,...,\n- Fuels\n- Chemicals\n\n\nSpecific Product(s),\n- Fuels\n\n\nSpecific Product(s),\n' Chemicals\n\n\nSpecific Product(s),\n- Food/Feed\n- Chemicals\n- Biomass\n\n\nSpecific Product(s),\n- Food/Feed\n- Biomass\n\n\nSpecific Product(s),\n- Consumer goods\n- Biomass\n\n\nSpecific Product(s),\n- Captured CO2\n\n\nSpecific Product(s),\n- (Building) Materials\n\n\nSpecific Product(s),\n- Consumer goods\n- Chemicals\n\n\nSpecific Product(s),ProjectName
0,This project aims mainly at demonstrating DMXT...,Europe,Timeline Start - End:\n2019-05-01 - 2023-04-30...,http://cordis.europa.eu/project/id/838031,Activities:\n- Life cycle assessment (LCA)\n- ...,CO2 capture/Utilization:\n\n\nProduction Volum...,Product Categories:\n- Captured CO2\n\n\nSpeci...,"TOTAL, AMAL, BREVIK, CMI, Axens, UETIKON, Gass...",This project aims mainly at demonstrating DMXT...,IFPEN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3D-Project DMX
1,The global aviation sector has set a course to...,Europe,Timeline Start - End:\n2021-05-01 - 2024-04-30...,http://www.4aircraft-project.eu/,CCU Tecnology Category:\nUtilization\n\n\nInfo...,CO2 capture/Utilization:\n\n\nProduction Volume:,Product Categories:\n- Fuels\n\n\nSpecific Pro...,"UNIVERSIDADE DE SAO PAULO, UNIVERSITEIT ANTWER...",The global aviation sector has set a course to...,FUNDACION PARA EL DESARROLLO DE LAS NUEVAS TEC...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4AirCRAFT
2,This business-oriented project aims at a new a...,Europe,Timeline Start - End:\n2017-01-05 - 2020-07-31...,http://www.abacus-bbi.eu/,Activities:\n- Techno-economic analysis\n- R&I...,CO2 capture/Utilization:\n\n\nProduction Volume:,Product Categories:\n- Food/Feed\n- Chemicals\...,"Sensient Cosmetic Technologies, MICROPHYT, Pro...",This business-oriented project aims at a new a...,Alternative Energies and Atomic Energy Commiss...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ABACUS


Lets now see how many missing values there is per column

In [25]:
# Identify and calculate the percentage of the missing values in each attribute

missig_values = testi_12.isnull().sum() / testi_12.shape[0] * 100

# Create a DataFrame from the Series
missig_values_df = pd.DataFrame(missig_values, columns=['Percentage Missing'])

_0_19_missing = missig_values_df[missig_values_df["Percentage Missing"].between(0, 19)]
_20_39_missing = missig_values_df[missig_values_df["Percentage Missing"].between(20, 39)]
_40_59_missing = missig_values_df[missig_values_df["Percentage Missing"].between(40, 59)]
_60_79_missing = missig_values_df[missig_values_df["Percentage Missing"].between(60, 79)]
_80_100_missing = missig_values_df[missig_values_df["Percentage Missing"].between(80, 100)]

In [26]:
_0_19_missing

,Percentage Missing
Project Description,0.000000
Region,2.734375
Project Boundaries,0.000000
Website,0.000000
Technological Information,0.000000
Technological Details,0.000000
Product Information,0.000000
Partners,5.859375
ProjectDescription,0.000000
ProjectLead,0.000000


In [27]:
_20_39_missing

,Percentage Missing
Country,30.078125
EndTime,22.265625
ProjectLength,22.656250


In [28]:
_40_59_missing

,Percentage Missing
SpecificProductsA,40.234375


In [29]:
_60_79_missing

,Percentage Missing
Start TRL,73.046875
CO2 capture/Utilization,78.515625
Production Volume,78.906250


In [30]:
_80_100_missing

,Percentage Missing
Trash,100.000000
Specific Product(s),96.093750
SpecificProductsB,98.828125
\n- Chemicals\n\n\nSpecific Product(s),92.187500
\n- Consumer goods\n- Chemicals\n- Biomass\n\n\nSpecific Product(s),99.609375
\n- Biomass\n\n\nSpecific Product(s),98.046875
\n- Consumer goods\n\n\nSpecific Product(s),99.609375
\n- Fuels\n- Chemicals\n\n\nSpecific Product(s),99.218750
\n- Fuels\n\n\nSpecific Product(s),99.218750
\n' Chemicals\n\n\nSpecific Product(s),99.609375


Then lets rename the columns that needs renaming to be more easily taken into use

In [37]:
testi_12.rename(columns={'CO2 Source':'CO2_Source'}, inplace=True)
testi_12.rename(columns={'CCU Tecnology Category':'CCU_Tecnology_Category'}, inplace=True)
testi_12.rename(columns={'End TRL':'End_TRL'}, inplace=True)
testi_12.rename(columns={'Product Categories':'Product_Categories'}, inplace=True)

testi_12.rename(columns={'SpecificProductsA':'Specific_Products_A'}, inplace=True)

testi_12.rename(columns={'Start TRL':'Start_TRL'}, inplace=True)
testi_12.rename(columns={'CO2 capture/Utilization':'CO2_Capture_Utilization'}, inplace=True)
testi_12.rename(columns={'Production Volume':'Production_Volume'}, inplace=True)

testi_12.rename(columns={'SpecificProductsB':'Specific_Products_B'}, inplace=True)
testi_12.rename(columns={'Specific Product(s)':'Specific_Products_C'}, inplace=True)

testi_12.rename(columns={'\n- Chemicals\n\n\nSpecific Product(s)':'Specific_Products_D'}, inplace=True)
testi_12.rename(columns={'\n- Consumer goods\n- Chemicals\n- Biomass\n\n\nSpecific Product(s)':'Specific_Products_E'}, inplace=True)
testi_12.rename(columns={'\n- Biomass\n\n\nSpecific Product(s)':'Specific_Products_F'}, inplace=True)
testi_12.rename(columns={'\n- Consumer goods\n\n\nSpecific Product(s)':'Specific_Products_G'}, inplace=True)
testi_12.rename(columns={'\n- Fuels\n- Chemicals\n\n\nSpecific Product(s)':'Specific_Products_H'}, inplace=True)
testi_12.rename(columns={'\n- Fuels\n\n\nSpecific Product(s)':'Specific_Products_I'}, inplace=True)
testi_12.rename(columns={'\n- Chemicals\n\n\nSpecific Product(s)':'Specific_Products_J'}, inplace=True)
testi_12.rename(columns={'\n- Food/Feed\n- Chemicals\n- Biomass\n\n\nSpecific Product(s)':'Specific_Products_K'}, inplace=True)
testi_12.rename(columns={'\n- Food/Feed\n- Biomass\n\n\nSpecific Product(s)':'Specific_Products_L'}, inplace=True)
testi_12.rename(columns={'\n- Consumer goods\n- Biomass\n\n\nSpecific Product(s)':'Specific_Products_M'}, inplace=True)
testi_12.rename(columns={'\n- Captured CO2\n\n\nSpecific Product(s)':'Specific_Products_N'}, inplace=True)
testi_12.rename(columns={'\n- (Building) Materials\n\n\nSpecific Product(s)':'Specific_Products_O'}, inplace=True)
testi_12.rename(columns={'\n- Consumer goods\n- Chemicals\n\n\nSpecific Product(s)':'Specific_Products_P'}, inplace=True)

Now we can drop off the columns that we no longer need in the dataframe. These columns have given values to the columns that stay in the dataframe

In [32]:
testi_12 = testi_12.drop("Project Description", axis=1)
testi_12 = testi_12.drop("Project Boundaries", axis=1)
testi_12 = testi_12.drop("Technological Information", axis=1)
testi_12 = testi_12.drop("Technological Details", axis=1)
testi_12 = testi_12.drop("Product Information", axis=1)
testi_12 = testi_12.drop("Trash", axis=1)

In [34]:
print(f"There is {testi_12.shape[1]} columns and {testi_12.shape[0]} rows in the testi_12 DataFrame now")

There is 39 columns and 256 rows in the testi_12 DataFrame now


As a final step we can now save this dataframe to a CSV file

In [35]:
testi_12.to_csv('co2_value_europe_database.csv', index=False)

In [40]:
testi_12.columns

Index(['Region', 'Website', 'Partners', 'ProjectDescription', 'ProjectLead',
       'Country', 'StartTime', 'EndTime', 'Status', 'Budget', 'FundingSource',
       'ProjectLength', 'Activities', 'CO2_Source', 'CCU_Tecnology_Category',
       'Facility stage', 'Start_TRL', 'End_TRL', 'Info On Utilization',
       'CO2_Capture_Utilization', 'Production_Volume', 'Product_Categories',
       'Specific_Products_C', 'Specific_Products_A', 'Specific_Products_B',
       'Specific_Products_D', 'Specific_Products_E', 'Specific_Products_F',
       'Specific_Products_G', 'Specific_Products_H', 'Specific_Products_I',
       '\n' Chemicals\n\n\nSpecific Product(s)', 'Specific_Products_K',
       'Specific_Products_L', 'Specific_Products_M', 'Specific_Products_N',
       'Specific_Products_O', 'Specific_Products_P', 'ProjectName'],
      dtype='object')